In [1]:
import os
from os import path
import numpy as np
import random
import copy
import pandas as pd
import ipython_bell

from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from astropy.visualization import make_lupton_rgb
import sys

In [2]:
#Makes 3 plots per image: source simulation, lens cutout, complete simulation
def make_graphs(name, sim_i, sim_r, sim_g, cutout_i, cutout_r, cutout_g):
    loc = plticker.MultipleLocator(base=3.75)
    fig, (ax1, ax2, ax3) = plt.subplots(figsize=(15, 4), ncols=3)

    rgb = make_lupton_rgb(sim_i, sim_r, sim_g, Q=11., stretch=40.)
    ax1.set_title('Source(s)')
    ax1.xaxis.set_ticklabels([])
    ax1.yaxis.set_ticklabels([])
    #ax1.xaxis.set_major_locator(loc)
    #ax1.yaxis.set_major_locator(loc)
    #ax1.grid(which='major', axis='both', linestyle='-')
    original = ax1.imshow(rgb, aspect='equal')#, vmin=0, vmax = 250)
    #fig.colorbar(original, ax=ax1)

    rgb = make_lupton_rgb(cutout_i, cutout_r, cutout_g, Q=11., stretch=40.)
    ax2.set_title('Lens')
    ax2.xaxis.set_ticklabels([])
    ax2.yaxis.set_ticklabels([])
    #ax2.xaxis.set_major_locator(loc)
    #ax2.yaxis.set_major_locator(loc)
    #ax2.grid(which='major', axis='both', linestyle='-')
    cutout = ax2.imshow(rgb, aspect='equal')
    #fig.colorbar(cutout, ax=ax2)
            
    rgb = make_lupton_rgb(sim_i+cutout_i, sim_r+cutout_r, sim_g+cutout_g, Q=11., stretch=40.)
    ax3.set_title('Complete Simulation')
    ax3.xaxis.set_ticklabels([])
    ax3.yaxis.set_ticklabels([])
    #ax3.xaxis.set_major_locator(loc)
    #ax3.yaxis.set_major_locator(loc)
    #ax3.grid(which='major', axis='both', linestyle='-')
    projection = ax3.imshow(rgb, aspect='equal')
    #fig.colorbar(projection, ax=ax3)
    sub_path = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/'
    #plt.savefig(sub_path + 'Data/Sim_complete/Image' + name + '.png', bbox_inches='tight')
    plt.show()

In [3]:
# Writes fit file
def write_fit_file(name, x, datos):
    primary = fits.PrimaryHDU()
    image = fits.ImageHDU(x, name="IMAGE")
    #table = fits.TableHDU(data = datos)
    table = fits.BinTableHDU(data = datos)
    hdu_list = fits.HDUList([primary, image, table])
    hdu_list.writeto('/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/Sim_complete/' + name + '.fits', overwrite=True)

In [4]:
def complete_simulations(m, ids):
    directory_sim = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/Sim/'
    directory_cutouts = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/lens_cutouts/'
    end_name = ['_g.fits', '_i.fits', '_r.fits']
    x = np.zeros((1,3,46,46)) #Complete Simulations
    n = 0
    for lens_id in ids:
        path_sim_id = directory_sim + str(lens_id)
        if(n >= m):
            break
        #check that the sim id folder exist (if not continue)
        if(os.path.exists(path_sim_id) == False):
            continue
        num_sim_id = int(len(os.listdir(path_sim_id))/3) # num sim with same lens
        #loop on each simulation produced using the same lens
        for j in range(1, num_sim_id+1):
            n += 1 #number of total simulations
            path_cutout = directory_cutouts + str(lens_id) 
            path_sim = path_sim_id + '/' + str(j)
            if(os.path.exists(path_cutout + end_name[0]) == False): 
                continue
            with fits.open(path_cutout + end_name[0]) as hdul: cutout_g = hdul[0].data
            with fits.open(path_cutout + end_name[1]) as hdul: cutout_i = hdul[0].data
            with fits.open(path_cutout + end_name[2]) as hdul: cutout_r = hdul[0].data
            with fits.open(path_sim + end_name[0]) as hdul: sim_g = hdul[0].data
            with fits.open(path_sim + end_name[1]) as hdul: sim_i = hdul[0].data
            with fits.open(path_sim + end_name[2]) as hdul: sim_r = hdul[0].data
            
            ele_sim = np.array([cutout_g + sim_g, cutout_r + sim_r, cutout_i + sim_i])
            ele_sim = np.rot90(ele_sim, k=random.randint(0, 4), axes=(1, 2))
            if(random.randint(0, 1) == 1):
                ele_sim = np.flip(ele_sim, 2)
            x = np.append(x, [ele_sim], axis = 0)
    x = np.delete(x, 0, axis = 0)
    return(x)

In [5]:
#19712 lens cutouts
ids = pd.read_csv('/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/all_data_sim.csv')['COADD_OBJECT_ID'].sample(frac = 1)
x = complete_simulations(10, ids)
print(len(x))

10


In [6]:
#generating table:

datos = np.zeros((1,4))
ele_sim = np.array([1,2,3,4])

for i in range(3):
    datos = np.append(datos, np.ones((1,4)), axis = 0)
    datos = np.append(datos, [ele_sim], axis = 0)  
    datos = np.append(datos, np.zeros((1,4)), axis = 0)

print(datos)

names = ['a', 'b', 'c', 'd']
datos = Table(data = datos, names = names)
datos

[[0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [1. 2. 3. 4.]
 [0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [1. 2. 3. 4.]
 [0. 0. 0. 0.]
 [1. 1. 1. 1.]
 [1. 2. 3. 4.]
 [0. 0. 0. 0.]]


a,b,c,d
float64,float64,float64,float64
0.0,0.0,0.0,0.0
1.0,1.0,1.0,1.0
1.0,2.0,3.0,4.0
0.0,0.0,0.0,0.0
1.0,1.0,1.0,1.0
1.0,2.0,3.0,4.0
0.0,0.0,0.0,0.0
1.0,1.0,1.0,1.0
1.0,2.0,3.0,4.0


In [7]:
name_file = 'other' #'other'

In [8]:
print(x.shape)
write_fit_file(name_file, x, datos)

(10, 3, 46, 46)


In [9]:
# convert -delay 140 -loop 0 *.png Complete_cut_sim.gif

In [10]:
filename = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/Sim_complete/'
filename = filename + 'other.fits'
hdul = fits.open(filename)
hdul.info()

Filename: /Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/Sim_complete/other.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  IMAGE         1 ImageHDU        10   (46, 46, 3, 10)   float64   
  2                1 BinTableHDU     16   10R x 4C   [D, D, D, D]   


In [25]:
data = hdul[2].data
df = pd.DataFrame(hdul[2].data)
df

,a,b,c,d
0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0
2,1.0,2.0,3.0,4.0
3,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0
5,1.0,2.0,3.0,4.0
6,0.0,0.0,0.0,0.0
7,1.0,1.0,1.0,1.0
8,1.0,2.0,3.0,4.0
9,0.0,0.0,0.0,0.0
